In [ ]:
import subprocess

english_list = ['One', 'Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine'] + [str(i) for i in range(1, 10)]

def generate_intNumber(question_str):
    if question_str.isdigit():
        return int(question_str)
    else:
        return english_list.index(question_str) + 1

def find_question(question):
    _question = question.split()
    if _question[0] in english_list:
        return(question)
    else:
        first_word = ''
        for eng in english_list:
            for qu in _question:
                index = qu.find(eng)
                if index > 0:
                    first_word = eng
                    index = question.find(first_word)
                    return question[index:]


# build communication with app_mining
#proc = subprocess.Popen(["python3" , "question.py"], stdin=subprocess.PIPE, stdout=subprocess.PIPE)
proc = subprocess.Popen(["./app_mining" , "b0829020"], stdin=subprocess.PIPE, stdout=subprocess.PIPE)
# capture question and answer
question_list = []
word = ""
while True:
    # capture question
    out = proc.stdout.read(1).decode()
    if out != "\n":
        word += out
        if "=" in word:
            print(word)
            word = find_question(word)
            question_list = word.split()
            
            num1 = generate_intNumber(question_list[0]) 
            num2 = generate_intNumber(question_list[2])
            answer = num1 * num2
            print(f"{num1} * {num2} = ", answer)

            # passing answer
            proc.stdin.write(f"{answer}\n".encode())
            proc.stdin.flush()
            word = ""
    else:
        print(word)
        word = ""
proc.communicate()


In [ ]:
import random

english_list = ['One', 'Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine'] + list(range(1,10))

for i in range(0, 10):
    num1 = random.randint(0,8)
    num2 = random.randint(0,8)
    english_or_number = random.randint(0, 1)

    if english_or_number: # 英文
        print(f"{english_list[num1]} * {english_list[num2]} = ", end="")
    else:
        print(f"{num1+1} * {num2+1} = ", end="")

    answer = int(input())
    if answer == (num1 + 1)* (num2 + 1):
        print("Correct, you got 42 seconds", answer)
        print("mining bonus: 0.09\n")
    else:
        break

In [ ]:
question_list = ["7 x 8 x 6 x 4 is ", "5 * 4 = ", "What is 4 * 3 : ", "What is 6  x  8 x 7  x  7: ", "   Please answer 1 * 4 * 6 = ", "Three * Four ="]
answer_list = [1344, 20, 12, 2352, 24, 12]

for qu, ans in zip(question_list, answer_list):
    print(qu, end="")
    ans_input = int(input())
    if ans_input == ans:
        print("Correct!")
    else:
        print("Sorry!")

In [ ]:
import subprocess
import threading
import operator

ops = {
    '+' : operator.add, '-' : operator.sub, 
    '*' : operator.mul, 'x' : operator.mul, '/' : operator.truediv,
    '%' : operator.mod, '^' : operator.xor,
}
english_list = ['One', 'Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine'] + [str(i) for i in range(1, 10)]

def generate_intNumber(question_str):
    if question_str.isdigit():
        return int(question_str)
    else:
        return english_list.index(question_str) + 1
def find_question(question):
    # find the start index in question
    _question = question.split()
    ques = ''
    for qu in _question: 
        if ques:
            break
        for eng in english_list:
            index = qu.find(eng)
            if index > -1:
                first_word = eng
                index = question.find(first_word)
                ques = question[index:]
                break
    # find the whole question
    question_list = []
    _question = ques.split()
    for index, qu in enumerate(_question):
        if qu in ops:
            aft_operator = _question[index+1]
            question_list.extend([_question[index - 1], qu])
    if len(question_list) > 0:
        aft_operator = aft_operator.translate(str.maketrans('', '', ",:"))
        question_list.extend(aft_operator)
        return question_list

# build communication with app_mining
def communication(a):
    
    proc = subprocess.Popen(["python3" , "question.py"], stdin=subprocess.PIPE, stdout=subprocess.PIPE)
    #proc = subprocess.Popen(["./app_mining" , "b0829020"], stdin=subprocess.PIPE, stdout=subprocess.PIPE)
    print("*******start**********")
    # capture question and answer
    word = ""
    while a==True:
        # capture question
        out = proc.stdout.read(1).decode()
        if out != "\n":
            word += out
            if "=" in word or ":" in word:
                print(word)
                question_list = find_question(word)
                
                if question_list > 0:
                    ans = generate_intNumber(question_list[0])
                    for index, word in enumerate(question_list):
                        if word in ops:
                            ans = ops[word](ans, generate_intNumber(question_list[index + 1] ))
                    print(f"ans: {answer}")
                    # passing answer  
                    proc.stdin.write(f"{answer}\n".encode())
                    proc.stdin.flush()
            elif "Sorry" in word:
                proc = subprocess.Popen(["python3" , "question.py"], stdin=subprocess.PIPE, stdout=subprocess.PIPE)
                #proc = subprocess.Popen(["./app_mining" , "b0829020"], stdin=subprocess.PIPE, stdout=subprocess.PIPE)
                print("*****restart!********")
                word = ""
        else:
            print(word)
            word = ""
    proc.communicate()

num = []
t_list = []

for n in range(1):
    name = ("name", n)
    num.append(name)
for number in num:
    number = threading.Thread(target = communication, args=(True,))
    t_list.append(number)

# 開始工作
for t in t_list:
    t.start()
 
# 調整多程順序
for t in t_list:
    t.join()

In [ ]:
import subprocess
import select
import operator


ops = {
    '+' : operator.add, '-' : operator.sub, 
    '*' : operator.mul, 'x' : operator.mul, '/' : operator.truediv,
    '%' : operator.mod, '^' : operator.xor,
}
english_list = ['One', 'Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine'] + [str(i) for i in range(1, 10)]

def generate_intNumber(question_str):
    if question_str.isdigit():
        return int(question_str)
    else:
        return english_list.index(question_str) + 1
def find_question(question):
    # find the start index in question
    _question = question.split()
    ques = ''
    for qu in _question: 
        if ques:
            break
        for eng in english_list:
            index = qu.find(eng)
            if index > -1:
                first_word = eng
                index = question.find(first_word)
                ques = question[index:]
                break
    # find the whole question
    question_list = []
    _question = ques.split()
    for index, qu in enumerate(_question):
        if qu in ops:
            aft_operator = _question[index+1]
            question_list.extend([_question[index - 1], qu])
    if len(question_list) > 0:
        aft_operator = aft_operator.translate(str.maketrans('', '', ",:"))
        question_list.extend(aft_operator)
        return question_list


# build communication with app_mining
def communication():
    
    proc = subprocess.Popen(["python3" , "question.py"], stdin=subprocess.PIPE, stdout=subprocess.PIPE)
    #proc = subprocess.Popen(["./app_mining" , "b0829020"], stdin=subprocess.PIPE, stdout=subprocess.PIPE)
    print("*******start**********")
    # capture question and answer
    word = ""
    while True:
        # capture question
        out = proc.stdout.read(1).decode()
        
        if out != "\n":
            word += out
            
            
        else:
            print("else: ", word)
            question_list = find_question(word)
                
            if question_list:
                answer = generate_intNumber(question_list[0])
                for index, word in enumerate(question_list):
                    if word in ops:
                        answer = ops[word](answer, generate_intNumber(question_list[index + 1] ))
                print(f"ans: {answer}")
                # passing answer  
                proc.stdin.write(f"{answer}\n".encode())
                proc.stdin.flush()
            elif "Sorry" in word:
                #proc = subprocess.Popen(["./app_mining" , "b0829020"], stdin=subprocess.PIPE, stdout=subprocess.PIPE)
                print("*****restart!********")
                word = ""
            word = ""
            
    proc.communicate()

communication()

In [ ]:
import subprocess
import operator


ops = {
    '+' : operator.add, '-' : operator.sub, 
    '*' : operator.mul, 'x' : operator.mul, '/' : operator.truediv,
    '%' : operator.mod, '^' : operator.xor,
}
english_list = ['One', 'Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine'] + [str(i) for i in range(1, 10)]

def generate_intNumber(question_str):
    if question_str.isdigit():
        return int(question_str)
    else:
        return english_list.index(question_str) + 1
def find_question(question):
    # find the start index in question
    _question = question.split()
    ques = ''
    for qu in _question: 
        if ques:
            break
        for eng in english_list:
            print(qu)
            index = qu.find(eng)
            if index > -1:
                first_word = eng
                index = question.find(first_word)
                ques = question[index:]
                break
    # find the whole question
    question_list = []
    _question = ques.split()
    for index, qu in enumerate(_question):
        if qu in ops:
            aft_operator = _question[index+1]
            question_list.extend([_question[index - 1], qu])
    if len(question_list) > 0:
        aft_operator = aft_operator.translate(str.maketrans('', '', ",':"))
        question_list.extend(aft_operator)
        return question_list

proc = subprocess.Popen(["python3" , "question.py"], stdin=subprocess.PIPE, stdout=subprocess.PIPE)

while True:
    outs = proc.communicate(str.encode("utf-8"), timeout=2)
    word = outs[0].decode('utf-8')
    question_list = find_question(word)
    if question_list:
        answer = generate_intNumber(question_list[0])
        for index, word in enumerate(question_list):
            if word in ops:
                answer = ops[word](answer, generate_intNumber(question_list[index + 1] ))
            print(f"ans: {answer}")
            # passing answer  
            proc.stdin.write(f"{answer}\n".encode())
            proc.stdin.flush()

    


In [ ]:
#from multiprocessing import process
import subprocess
import threading
#import concurrent.futures


english_list = ['One', 'Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine'] + [str(i) for i in range(1, 10)]


def generate_intNumber(question_str):
    if question_str.isdigit():
        return int(question_str)
    else:
        return english_list.index(question_str) + 1

def find_question(question):
    _question = question.split()
    if _question[0] in english_list:
        return(question)
    else:
        first_word = ''
        for eng in english_list:
            for qu in _question:
                index = qu.find(eng)
                if index > 0:
                    first_word = eng
                    index = question.find(first_word)
                    return question[index:]

def communication(a):
    # 建立與app_mining的通訊//is
    #proc = subprocess.Popen(["python3" , "question.py"], stdin=subprocess.PIPE, stdout=subprocess.PIPE)
    proc = subprocess.Popen(["./app_mining" , "b0829020"], stdin=subprocess.PIPE, stdout=subprocess.PIPE)
    print("start")
    # 重複讀取問題與回答
    question_list = []
    word = ""
    while a==True:
        # 讀取問題
        out = proc.stdout.read(1).decode()
        if out != "\n":
            word += out
            if "= " in word or ": " in word :
                calcu = 0
                print("question ",word)
                # word = find_question(word)
                question_list = word.split(" ")
                # print(question_list)
                answer = 1
                for i in range(0,len(question_list)):
                    if i == len(question_list)-2 :
                        if question_list[i][0].isdigit():
                            answer = answer*int(question_list[i][0])
                    for j in english_list:
                        if question_list[i]==j:
                            num = generate_intNumber(question_list[i])
                            answer = answer * num
                            calcu = 1 
                            # print(f"{num} , {answer}")  
                # 將答案傳給執行檔
                if calcu == 0:
                    word = ""
                else:
                    print("answer: ",answer)
                    proc.stdin.write(f"{answer}\n".encode())
                    proc.stdin.flush()
                    word = ""
            elif "Sorry" in word :
                print(word)
                proc = subprocess.Popen(["./app_mining" , "b0829020"], stdin=subprocess.PIPE, stdout=subprocess.PIPE)
                print("restart")
                word=""
        else:
            print(word)
            word = ""
    proc.communicate()

num = []
t_list = []

for n in range(1):
    name = ("name", n)
    num.append(name)
for number in num:
    number = threading.Thread(target = communication, args=(True,))
    t_list.append(number)

 
# 開始工作
for t in t_list:
    t.start()
 
# 調整多程順序
for t in t_list:
    t.join()

#with concurrent.futures.ThreadPoolExecutor(max_workers=500) as executor:
#    executor.map(communication(True))